In [ ]:
!pip install transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.5/7.5 MB 17.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 19.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 15.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 22.9 MB/s eta 0:00:00


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import pandas as pd
import numpy as np
import torch
from tqdm.auto import tqdm
import random
import os

def reset_seeds(seed):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True


DATA_PATH = "/content/drive/MyDrive/파이널프로젝트/data/"
SEED = 42

device = 'cuda' if torch.cuda.is_available() else 'cpu'
device

'cpu'

In [ ]:
df = pd.read_csv(f"{DATA_PATH}아모레크롤링_완료_필터링_수정.csv")
df

,상품분류,브랜드명,상품명,사용자 이름,별점,리뷰작성날짜,나이,성별,피부타입,피부트러블,리뷰
0,스킨케어,설화수,나의 첫 설화수,chob****,5,2023.08.13,30대,여성,복합성,민감성,"엄마가 좋아하는 에센스에요! 한방향나는 것도 좋고, 크림보단 에센스 좋은제품으로 바..."
1,스킨케어,설화수,나의 첫 설화수,moyr*,5,2023.08.13,50대 이상,여성,건성,탄력없음,더운 여름에는 윤조에센스 한 가지만 발라도 넘 촉촉합니다 이 나이에 피부 좋다는 소...
2,스킨케어,설화수,나의 첫 설화수,bobl*****,5,2023.08.13,50대 이상,여성,복합성,민감성,윤조에센스는 저의 가장 최애템이예요. 지금까지 수년간 지속적으로 빠짐없이 챙겨바르는...
3,스킨케어,설화수,나의 첫 설화수,mije******,5,2023.08.12,40대,여성,건성,주름,받아보니좋아요사진올리려했는데 안되서아쉽네요
4,스킨케어,설화수,나의 첫 설화수,hell****,5,2023.08.12,30대,여성,복합성,칙칙함,패키지 바뀌고 처음 사봤는데 좀더 세련되고 예쁜 느낌이에요 사계절 다 좋지만 여름에...
...,...,...,...,...,...,...,...,...,...,...,...
37813,뷰티푸드,바이탈뷰티,슈퍼콜라겐 구미 5입,juny*,5,2023.04.27,40대,여성,복합성,건조함,평소 아모레퍼시픽 바이탈뷰티의 콜라겐을 꾸준히 섭취하고 있는데 이번에 콜라겐 젤리가...
37814,뷰티푸드,바이탈뷰티,슈퍼콜라겐 구미 5입,spor***,5,2023.04.27,40대,여성,건성,칙칙함,신상품이라 구매해봤어요 먹기좋고 맛이 좋아 간식으로 먹으면 좋을거같아요 입심심할때...
37815,뷰티푸드,바이탈뷰티,슈퍼콜라겐 구미 5입,a010********,5,2023.04.21,40대,여성,건성,주름,애들이 더 좋아해서 재구매했어요 곤약젤리 느낌이랑 똑같다고 좋아하네요 애들이랑 함...
37816,뷰티푸드,바이탈뷰티,슈퍼콜라겐 구미 5입,Pny0***,5,2023.04.20,30대,여성,복합성,주름,먹기 편하고 맛있어요 간식먹고싶을때 먹는데 피부까지 좋아진다니 계속 시켜먹어야겠어요...


In [ ]:
! pip install keybert

  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.0/86.0 kB 2.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 19.1 MB/s eta 0:00:00
  Created wheel for keybert: filename=keybert-0.7.0-py3-none-any.whl size=23767 sha256=a0c3345fefe6689d8ee627a4517da6039a950e22fbac3d836503a19f82b8d89d
  Stored in directory: /root/.cache/pip/wheels/66/8d/e6/b0e2f8d883b0fd51819226f67ad9843e04913ce4a97241ff4b
  Created wheel for sentence-transformers: filename=sentence_transformers-2.2.2-py3-none-any.whl size=125924 sha256=2be03b792913d354b25c78a212861a79f417225350760e929bebe47e56fe6742
  Stored in directory: /root/.cache/pip/wheels/62/f2/10/1e606fd5f02395388f74e7462910fe851042f97238cbbd902f
Successfully built keybert sentence-transformers


In [ ]:
!pip install krwordrank

In [ ]:
def krword(title):
    from krwordrank.sentence import summarize_with_sentences
    from krwordrank.word import KRWordRank
    texts = df[df['상품명'] == title]['리뷰'].to_list()  # 넘파이 어레이를 리스트로 변환

    from krwordrank.hangle import normalize
    texts = [normalize(text, english=True, number=True) for text in texts]

    wordrank_extractor = KRWordRank(
        min_count=5,
        max_length=10,
        verbose=True
    )

    beta = 0.85
    max_iter = 10

    keywords, rank, graph = wordrank_extractor.extract(texts, beta, max_iter)

    for word, r in sorted(keywords.items(), key=lambda x: x[1], reverse=True)[:30]:
        print('%8s:\t%.4f' % (word, r))

In [ ]:
krword("나의 첫 설화수")

scan vocabs ... 
num vocabs = 738
done
     설화수:	8.4849
     좋아요:	8.2318
     좋어요:	5.1049
      너무:	5.0153
      제품:	4.8704
      윤조:	4.7493
      사용:	4.3954
     꾸준히:	3.4182
      구매:	3.3438
      피부:	3.3381
    촉촉하고:	3.0985
      좋은:	2.8380
   선물용으로:	2.7536
     6세대:	2.6995
    좋습니다:	2.6980
 아모레퍼시픽몰:	2.5534
      항상:	2.4203
    기획세트:	2.3970
      좋고:	2.3677
    공홈이라:	2.3088
   만독합니가:	2.1622
      계속:	2.1466
      아주:	2.1291
  만족스럽네요:	2.1090
   선믈용으로:	2.0880
     같아요:	2.0843
     에센스:	2.0358
      바로:	1.9586
     있어서:	1.8376
      자주:	1.8087


In [ ]:
!pip install konlpy

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 19.4/19.4 MB 34.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 465.3/465.3 kB 30.8 MB/s eta 0:00:00


In [ ]:
from konlpy.tag import Okt
from collections import Counter
from konlpy.tag import Twitter

In [ ]:
def kon(title):
  texts = df[df['상품명'] == "나의 첫 설화수"]['리뷰'].to_list()
  okt = Okt()
  nouns_list = []
  for text in texts:
    nouns = okt.nouns(text)
    nouns_list.extend(nouns)

  count = Counter(nouns_list)

  # 명사 빈도 카운트
  noun_list = count.most_common(100)
  for v in noun_list:
    if any(keyword in v[0] for keyword in ['지성', '건성', '복합성','여드름','탄력','모공','민감성']):
      print(v)

In [ ]:
kon("나의 첫 설화수")

('건성', 8)
('탄력', 6)


In [ ]:
texts = df[df['상품명'] == "나의 첫 설화수"]['리뷰'].to_list()

In [ ]:
okt = Okt()
nouns_list = []
for text in texts:
  nouns = okt.nouns(text)
  nouns_list.extend(nouns)

count = Counter(nouns_list)

# 명사 빈도 카운트
noun_list = count.most_common(100)
for v in noun_list:
  if any(keyword in v[0] for keyword in ['지성', '건성', '복합성','여드름','탄력','모공','민감성']):
	  print(v)

('건성', 8)
('탄력', 6)


In [ ]:
def keyword_extractor(tagger, text):
    tokens = tagger.phrases(text)
    tokens = [ token for token in tokens if len(token) > 1 ] # 한 글자인 단어는 제외
    count_dict = [(token, text.count(token)) for token in tokens ]
    ranked_words = sorted(count_dict, key=lambda x:x[1], reverse=True)[:10]
    return [ keyword for keyword, freq in ranked_words ]

if __name__ == '__main__':
    twit = Twitter()
    print( keyword_extractor(twit, text) )

['피부', '윤조', '자꾸', '좋다는 화장품', '늘 뭔가', '설화수', '화장품', '뭔가']


/usr/local/lib/python3.10/dist-packages/konlpy/tag/_okt.py:17: UserWarning: "Twitter" has changed to "Okt" since KoNLPy v0.4.5.
  warn('"Twitter" has changed to "Okt" since KoNLPy v0.4.5.')


In [ ]:
# Graph ranking 이 높은 노드들(substrings)이 후처리 과정을 거쳐 단어로 출력
for word, r in sorted(keyword.items(), key=lambda x:x[1], reverse=True)[:30]:
        print('%8s:\t%.4f' % (word, r))

NameError: ignored